<a href="https://colab.research.google.com/github/descartesmbogning/Marburg-virus-research/blob/main/3_analysis_gender_disparities_extract_all_authors_name.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import missingno as msno
%matplotlib inline
import csv

In [ ]:
# Set the figure size - handy for larger output
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = [6, 6]
# Set up with a higher resolution screen (useful on Mac)
%config InlineBackend.figure_format = 'retina'

In [ ]:
from scipy import stats #the only import you actually need for this
import matplotlib.pyplot as plt
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}

In [ ]:
#example function that tests distribution then picks test accordingly
def test_sig(x, y):
    if stats.shapiro(x).pvalue >= 0.05 and stats.shapiro(y).pvalue >= 0.05: #if they are normally distriuted
        print("t-test")
        return stats.ttest_ind(x,y)
    else:
        print("mann-whitney")
        return stats.mannwhitneyu(x, y)

# gender disparities

In [ ]:
from functools import reduce

#Import data
data = pd.read_excel(f'D:\MARBURG VIRUS DISEASES/data/WOSCC/woscc_bon_23_02_verif_on_wos.xls')[['UT (Unique WOS ID)',
                                                                                                'Publication Year',
                                                                                                'Funding Orgs',
                                                                                                'Author Full Names']];
# create a columns of funding yes or no
data['funding_yes'] = data['Funding Orgs'].apply(lambda x: 1 if not pd.isnull(x) else 0)#.to_frame().drop('Funding Orgs', axis=1)

# set accession number as index
data = data.set_index(['UT (Unique WOS ID)','Publication Year','Funding Orgs','funding_yes'])

#
df_expand = data['Author Full Names'].str.split('; ', expand=True)

# extract all the cells data
author_full_names_list = df_expand.stack().explode().droplevel(0).to_frame().reset_index()#.drop('index', axis=1)

###
author_full_names_list_count =author_full_names_list[0].value_counts().to_frame()

## extract firstname
author_full_names_list_count = author_full_names_list_count.reset_index()
author_full_names_list_count['firstname'] = author_full_names_list_count['index'].str.split(', ', expand=True)[[1]]

 #save data
author_full_names_list_count.to_csv(f'../DATA_PREPROCESSING/author_full_names_list_count_and_firstname.csv')

#print
author_full_names_list_count

,index,0,firstname
0,"Bavari, Sina",54,Sina
1,"Geisbert, Thomas W.",47,Thomas W.
2,"Feldmann, Heinz",44,Heinz
3,"Nichol, Stuart T.",33,Stuart T.
4,"Becker, Stephan",32,Stephan
...,...,...,...
4194,"Ivanov, Ivan",1,Ivan
4195,"Allegranzi, Benedetta",1,Benedetta
4196,"de La Vega, Marc-Antoine",1,Marc-Antoine
4197,"Soule, Geoff",1,Geoff


In [ ]:
data['Author Full Names'].count()

932

# Import data with gender infos

In [ ]:
# Import data with gender infos
all_author_full_names_list_and_gender = pd.read_excel('../DATA_PREPROCESSING/author_full_names_list_count_and_firstname_genderize.xlsx')
all_author_full_names_list_and_gender['full_name'] = all_author_full_names_list_and_gender['full_name'].str.lower()
all_author_full_names_list_and_gender

,Unnamed: 0,full_name,0,firstname,ga_first_name,ga_gender,ga_accuracy,ga_samples,Unnamed: 8
0,27,"muhlberger, e",15,E,NaN,unclassified,NaN,NaN,NaN
1,45,"schmaljohn, a",11,A,NaN,unclassified,NaN,NaN,NaN
2,57,"kolesnikova, l",10,L,NaN,unclassified,NaN,NaN,NaN
3,82,"borchert, m",8,M,NaN,unclassified,NaN,NaN,NaN
4,86,"slenczka, w",8,W,NaN,unclassified,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
4194,3214,"wei, shimeng",1,Shimeng,Shimeng,unknown,50.0,10.0,NaN
4195,2056,"westmoreland, susan v.",1,Susan V.,Susan V.,unknown,50.0,2.0,NaN
4196,3609,"yang, yi yan",1,Yi Yan,Yi Yan,unknown,50.0,2.0,NaN
4197,4009,"zhang, lihe",1,Lihe,Lihe,unknown,50.0,6.0,NaN


In [ ]:
# lowercase all the columns
data_author_lower = df_expand.applymap(lambda x: x.lower() if pd.notnull(x) else x)
data_author_lower

,,,,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
UT (Unique WOS ID),Publication Year,Funding Orgs,funding_yes,,,,,,,,,,,,,,,,,,,,,
WOS:000904661800001,2023.0,NaN,0,"bi, jinhao","wang, haojie","han, qiuxue","pei, hongyan","wang, hualei","jin, hongli","jin, song","chi, hang","yang, songtao","zhao, yongkun",...,None,None,None,None,None,None,None,None,None,None
WOS:000921279800001,2023.0,National Institutes of Health (NIH) [U19AI142785]; NIH [UC7AI094660],1,"ye, xin","holland, richard","wood, mark","pasetka, chris","palmer, lorne","samaridou, eleni","mcclintock, kevin","borisevich, viktoriya","geisbert, thomas w.","cross, robert w.",...,None,None,None,None,None,None,None,None,None,None
WOS:000909001200001,2023.0,Russian Science Foundation; [22-13-00036],1,"moskalik, mikhail yu.",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
WOS:000777045900001,2022.0,National Research Foundation of Korea (NRF) - Ministry of Education [NRF-2020R1I1A2066868]; National Research Foundation of Korea (NRF) - Korean government (MSIT) [2020R1A5A2019413],1,"abir, mehedy hasan","rahman, tanjilur","das, ayan","etu, silvia naznin","nafiz, iqbal hossain","rakib, ahmed","mitra, saikat","bin emran, talha","dhama, kuldeep","islam, ariful",...,None,None,None,None,None,None,None,None,None,None
WOS:000849602200001,2022.0,NaN,0,"sah, ranjit","mohanty, aroop","reda, abdullah","siddiq, abdelmonem","mohapatra, ranjan k.","dhama, kuldeep",None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WOS:A1968B358900049,1968.0,NaN,0,"malherbe, h",strickla.m,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
WOS:A1968A855300001,1968.0,NaN,0,"martini, ga","knauff, hg","schmidt, ha","mayer, g","baltzer, g",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
WOS:A1968A855300006,1968.0,NaN,0,"siegert, r","shu, hl","slenczka, w",None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
name_list = all_author_full_names_list_and_gender['full_name']#.to_list()
name_list[:20]

0         muhlberger, e
1         schmaljohn, a
2        kolesnikova, l
3           borchert, m
4           slenczka, w
5             kiley, mp
6              hevey, m
7             negley, d
8            sanchez, a
9         mccormick, jb
10       colebunders, r
11             lloyd, g
12       mulangu, sabue
13           stroher, u
14    van der stuyft, p
15              will, c
16           bamberg, s
17           blinov, vm
18            ellis, ds
19            gear, jhs
Name: full_name, dtype: object

In [ ]:
gender_list = all_author_full_names_list_and_gender['ga_gender']#.to_list()
gender_list[:20]

0     unclassified
1     unclassified
2     unclassified
3     unclassified
4     unclassified
5     unclassified
6     unclassified
7     unclassified
8     unclassified
9     unclassified
10    unclassified
11    unclassified
12    unclassified
13    unclassified
14    unclassified
15    unclassified
16    unclassified
17    unclassified
18    unclassified
19    unclassified
Name: ga_gender, dtype: object

In [ ]:
# using zip()
# to convert lists to dictionary
name_gender_keys_values = dict(zip(name_list, gender_list))
# name_gender_keys_values

In [ ]:
for col in data_author_lower.columns:
    data_author_lower[col] = data_author_lower[col].map(name_gender_keys_values)
data_author_lower

,,,,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
UT (Unique WOS ID),Publication Year,Funding Orgs,funding_yes,,,,,,,,,,,,,,,,,,,,,
WOS:000904661800001,2023.0,NaN,0,male,male,unclassified,female,male,unknown,male,unclassified,male,male,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOS:000921279800001,2023.0,National Institutes of Health (NIH) [U19AI142785]; NIH [UC7AI094660],1,male,male,male,male,male,female,male,female,male,male,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOS:000909001200001,2023.0,Russian Science Foundation; [22-13-00036],1,male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOS:000777045900001,2022.0,National Research Foundation of Korea (NRF) - Ministry of Education [NRF-2020R1I1A2066868]; National Research Foundation of Korea (NRF) - Korean government (MSIT) [2020R1A5A2019413],1,male,male,male,female,male,male,male,male,male,male,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOS:000849602200001,2022.0,NaN,0,male,male,male,male,male,male,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WOS:A1968B358900049,1968.0,NaN,0,unclassified,unclassified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOS:A1968A855300001,1968.0,NaN,0,unclassified,unclassified,unclassified,unclassified,unclassified,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WOS:A1968A855300006,1968.0,NaN,0,unclassified,unclassified,unclassified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
## get dummies to have contries on columns
dfs = [pd.get_dummies(data_author_lower[col]) for col in data_author_lower]
d = reduce(lambda x,y : x.add(y, fill_value=0), dfs)

# data cleaning
cols = [c for c in d.columns if c[-1:] != '.']
df=d[cols]
cols = [c for c in df.columns.to_list() if len(c) != 1]
data_author_lower_dummies=df[cols]
data_author_lower_dummies

,,,,female,male,unclassified,unknown
UT (Unique WOS ID),Publication Year,Funding Orgs,funding_yes,,,,
WOS:000904661800001,2023.0,NaN,0,2.0,7.0,3.0,1.0
WOS:000921279800001,2023.0,National Institutes of Health (NIH) [U19AI142785]; NIH [UC7AI094660],1,2.0,9.0,0.0,0.0
WOS:000909001200001,2023.0,Russian Science Foundation; [22-13-00036],1,0.0,1.0,0.0,0.0
WOS:000777045900001,2022.0,National Research Foundation of Korea (NRF) - Ministry of Education [NRF-2020R1I1A2066868]; National Research Foundation of Korea (NRF) - Korean government (MSIT) [2020R1A5A2019413],1,1.0,12.0,1.0,0.0
WOS:000849602200001,2022.0,NaN,0,0.0,6.0,0.0,0.0
...,...,...,...,...,...,...,...
WOS:A1968B358900049,1968.0,NaN,0,0.0,0.0,2.0,0.0
WOS:A1968A855300001,1968.0,NaN,0,0.0,0.0,5.0,0.0
WOS:A1968A855300006,1968.0,NaN,0,0.0,0.0,3.0,0.0


In [ ]:
data_author_lower_dummies['number_of_author'] = data_author_lower_dummies.sum(axis='columns')
data_author_lower_dummies['%_female'] = data_author_lower_dummies['female']/data_author_lower_dummies['number_of_author']*100
data_author_lower_dummies['%_male'] = data_author_lower_dummies['male']/data_author_lower_dummies['number_of_author']*100
# data_author_lower['number_of_unknown_or_unclassified'] = ((data_author_lower == '/') | (data_author_lower == 'unknown')).sum(axis=1)
data_author_lower_dummies['%_unclass_name'] = data_author_lower_dummies['unclassified']/data_author_lower_dummies['number_of_author']*100
data_author_lower_dummies['female_to_male_ratio'] = data_author_lower_dummies['female']/data_author_lower_dummies['male']
# data_author_lower['first_author_gender'] = data_author_lower[0]
# data_author_lower['first_author_gender'] =

#save data
data_author_lower_dummies.to_csv(f'../TABLES/author_gender_list_count_dummies.csv')
data_author_lower_dummies

,,,,female,male,unclassified,unknown,number_of_author,%_female,%_male,%_unclass_name,female_to_male_ratio
UT (Unique WOS ID),Publication Year,Funding Orgs,funding_yes,,,,,,,,,
WOS:000904661800001,2023.0,NaN,0,2.0,7.0,3.0,1.0,13.0,15.384615,53.846154,23.076923,0.285714
WOS:000921279800001,2023.0,National Institutes of Health (NIH) [U19AI142785]; NIH [UC7AI094660],1,2.0,9.0,0.0,0.0,11.0,18.181818,81.818182,0.000000,0.222222
WOS:000909001200001,2023.0,Russian Science Foundation; [22-13-00036],1,0.0,1.0,0.0,0.0,1.0,0.000000,100.000000,0.000000,0.000000
WOS:000777045900001,2022.0,National Research Foundation of Korea (NRF) - Ministry of Education [NRF-2020R1I1A2066868]; National Research Foundation of Korea (NRF) - Korean government (MSIT) [2020R1A5A2019413],1,1.0,12.0,1.0,0.0,14.0,7.142857,85.714286,7.142857,0.083333
WOS:000849602200001,2022.0,NaN,0,0.0,6.0,0.0,0.0,6.0,0.000000,100.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
WOS:A1968B358900049,1968.0,NaN,0,0.0,0.0,2.0,0.0,2.0,0.000000,0.000000,100.000000,NaN
WOS:A1968A855300001,1968.0,NaN,0,0.0,0.0,5.0,0.0,5.0,0.000000,0.000000,100.000000,NaN
WOS:A1968A855300006,1968.0,NaN,0,0.0,0.0,3.0,0.0,3.0,0.000000,0.000000,100.000000,NaN


In [ ]:
data_author_lower_dummies.describe()

,female,male,unclassified,unknown,number_of_author,%_female,%_male,%_unclass_name,female_to_male_ratio
count,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,853.000000
mean,2.244635,4.390558,0.860515,0.024678,7.520386,26.797780,55.899702,16.940186,inf
std,2.547976,4.514666,1.987573,0.168498,6.188640,25.032897,29.766127,32.725357,NaN
min,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,0.000000,3.000000,0.000000,39.615385,0.000000,0.000000
50%,1.500000,3.000000,0.000000,0.000000,6.000000,25.000000,58.333333,0.000000,0.428571
75%,3.000000,6.000000,1.000000,0.000000,10.000000,44.444444,75.000000,8.333333,1.000000
max,18.000000,58.000000,17.000000,2.000000,76.000000,100.000000,100.000000,100.000000,inf


## ALL

In [ ]:
from functools import reduce

#Import data
data = pd.read_excel(f'D:\MARBURG VIRUS DISEASES/data/WOSCC/woscc_bon_23_02_verif_on_wos.xls')[['UT (Unique WOS ID)',
                                                                                                'Publication Year',
                                                                                                'Funding Orgs',
                                                                                                'Author Full Names']];
# limit data to after 2007
data_2007 = data.reset_index()[data.reset_index()['Publication Year'] > 2006]
data_2007.head()

# create a columns of funding yes or no
data_2007['funding_yes'] = data_2007['Funding Orgs'].apply(lambda x: 1 if not pd.isnull(x) else 0)#.to_frame().drop('Funding Orgs', axis=1)

# set accession number as index
data_2007 = data_2007.set_index(['UT (Unique WOS ID)','Publication Year','Funding Orgs','funding_yes'])

#
df_expand_2007 = data_2007['Author Full Names'].str.split('; ', expand=True)

# extract all the cells data
author_full_names_list_2007 = df_expand_2007.stack().explode().droplevel(0).to_frame().reset_index()#.drop('index', axis=1)

###
author_full_names_list_count_2007 =author_full_names_list_2007[0].value_counts().to_frame()

## extract firstname
# author_full_names_list_count_2007 = author_full_names_list_count_2007.reset_index()
# author_full_names_list_count_2007['firstname'] = author_full_names_list_count_2007['index'].str.split(', ', expand=True)[[1]]

 #save data
author_full_names_list_count_2007.to_csv(f'../DATA_PREPROCESSING/author_full_names_list_count_and_firstname_2007.csv')

#print
author_full_names_list_count_2007

,0
"Bavari, Sina",52
"Geisbert, Thomas W.",46
"Feldmann, Heinz",42
"Becker, Stephan",32
"Basler, Christopher F.",31
...,...
"Cheptoyek, Leonard",1
"Wangila, Barasa Emmanuel",1
"Wanje, Michael",1
"Tusiime, Patrick",1


In [ ]:
author_full_names_list_count_2007 = author_full_names_list_count_2007.reset_index().rename(columns={0:'number_publication'})
author_full_names_list_count_2007['full_name'] = author_full_names_list_count_2007['index'].str.lower()
author_full_names_list_count_2007

,index,number_publication,full_name
0,"Bavari, Sina",52,"bavari, sina"
1,"Geisbert, Thomas W.",46,"geisbert, thomas w."
2,"Feldmann, Heinz",42,"feldmann, heinz"
3,"Becker, Stephan",32,"becker, stephan"
4,"Basler, Christopher F.",31,"basler, christopher f."
...,...,...,...
3588,"Cheptoyek, Leonard",1,"cheptoyek, leonard"
3589,"Wangila, Barasa Emmanuel",1,"wangila, barasa emmanuel"
3590,"Wanje, Michael",1,"wanje, michael"
3591,"Tusiime, Patrick",1,"tusiime, patrick"


In [ ]:
all_author_full_names_gender_2007_concat = author_full_names_list_count_2007.merge(all_author_full_names_list_and_gender, on = 'full_name', how='left')
all_author_full_names_gender_2007_concat

,index,number_publication,full_name,Unnamed: 0,0,firstname,ga_first_name,ga_gender,ga_accuracy,ga_samples,Unnamed: 8
0,"Bavari, Sina",52,"bavari, sina",0,54,Sina,Sina,male,63.0,5878.0,0
1,"Geisbert, Thomas W.",46,"geisbert, thomas w.",1,47,Thomas W.,Thomas W.,male,100.0,24.0,0
2,"Feldmann, Heinz",42,"feldmann, heinz",2,44,Heinz,Heinz,male,98.0,3977.0,0
3,"Becker, Stephan",32,"becker, stephan",4,32,Stephan,Stephan,male,99.0,19941.0,0
4,"Basler, Christopher F.",31,"basler, christopher f.",6,31,Christopher F.,Christopher,male,99.0,55041.0,0
...,...,...,...,...,...,...,...,...,...,...,...
3596,"Cheptoyek, Leonard",1,"cheptoyek, leonard",3754,1,Leonard,Leonard,male,98.0,7077.0,0
3597,"Wangila, Barasa Emmanuel",1,"wangila, barasa emmanuel",3755,1,Barasa Emmanuel,Barasa Emmanuel,male,99.0,43698.0,0
3598,"Wanje, Michael",1,"wanje, michael",3756,1,Michael,Michael,male,99.0,211741.0,0
3599,"Tusiime, Patrick",1,"tusiime, patrick",3757,1,Patrick,Patrick,male,99.0,118290.0,0


In [ ]:
all_author_full_names_gender_2007_concat['ga_gender'].value_counts()

male            2139
female          1303
unclassified     141
unknown           18
Name: ga_gender, dtype: int64

In [ ]:
all_author_full_names_gender_2007_concat['ga_gender'].value_counts(normalize=True)


male            0.594002
female          0.361844
unclassified    0.039156
unknown         0.004999
Name: ga_gender, dtype: float64

In [ ]:
# year_gender = data_author_lower_dummies.reset_index()[['UT (Unique WOS ID)','Publication Year','female','male','unclassified','unknown','number_of_author']
#                                                      ].groupby(['Publication Year']).agg({'female': 'sum', 'male': 'sum',
#                                                                                           'unclassified': 'sum', 'unknown': 'sum', 'number_of_author': 'sum',
#                                                                                           'UT (Unique WOS ID)': 'count'})

# year_gender['female_to_male_ratio'] = year_gender['female']/year_gender['male']
# year_gender['publication_to_authors_number_ratio'] = year_gender['number_of_author']/year_gender['UT (Unique WOS ID)']

# for col in ['female','male','unclassified','unknown']:
#     year_gender['%_'+col] = year_gender[col]/year_gender['number_of_author']*100

# # #save data
# year_gender.to_csv(f'../TABLES/year_gender_count.csv')
# year_gender

In [ ]:
# year_gender = data_author_lower_dummies[['UT (Unique WOS ID)','Publication Year','female','male','unclassified','unknown','number_of_author']].groupby(['Publication Year']).sum()#.drop('funding_yes',axis=1)
# year_gender['female_to_male_ratio'] = year_gender['female']/year_gender['male']

# for col in ['female','male','unclassified','unknown']:
#     year_gender['%_'+col] = year_gender[col]/year_gender['number_of_author']*100

# # #save data
# year_gender.to_csv(f'../DATA_PREPROCESSING/year_gender_count.csv')
# year_gender

In [ ]:
# fund_data_author_lower_dummies

## funding

In [ ]:
from functools import reduce

#Import data
data = pd.read_excel(f'D:\MARBURG VIRUS DISEASES/data/WOSCC/woscc_bon_23_02_verif_on_wos.xls')[['UT (Unique WOS ID)',
                                                                                                'Publication Year',
                                                                                                'Funding Orgs',
                                                                                                'Author Full Names']];
# limit data to after 2007
data_2007 = data.reset_index()[data.reset_index()['Publication Year'] > 2006]
data_2007.head()

# create a columns of funding yes or no
data_2007['funding_yes'] = data_2007['Funding Orgs'].apply(lambda x: 1 if not pd.isnull(x) else 0)#.to_frame().drop('Funding Orgs', axis=1)

data_2007 = data_2007.reset_index().loc[data_2007.reset_index()['funding_yes'] == 1]

# set accession number as index
data_2007 = data_2007.set_index(['UT (Unique WOS ID)','Publication Year','Funding Orgs','funding_yes'])

#
df_expand_2007 = data_2007['Author Full Names'].str.split('; ', expand=True)

# extract all the cells data
author_full_names_list_2007 = df_expand_2007.stack().explode().droplevel(0).to_frame().reset_index()#.drop('index', axis=1)

###
fund_author_full_names_list_count_2007 =author_full_names_list_2007[0].value_counts().to_frame()

## extract firstname
# author_full_names_list_count_2007 = author_full_names_list_count_2007.reset_index()
# author_full_names_list_count_2007['firstname'] = author_full_names_list_count_2007['index'].str.split(', ', expand=True)[[1]]

 #save data
fund_author_full_names_list_count_2007.to_csv(f'../DATA_PREPROCESSING/author_full_names_list_count_and_firstname_2007_fund.csv')

#print
fund_author_full_names_list_count_2007

,0
"Bavari, Sina",46
"Geisbert, Thomas W.",40
"Feldmann, Heinz",34
"Basler, Christopher F.",31
"Becker, Stephan",26
...,...
"Kwesiga, Benon",1
"Kofman, Aaron",1
"Masiira, Ben",1
"Brown, Shelley",1


In [ ]:
fund_author_full_names_list_count_2007 = fund_author_full_names_list_count_2007.reset_index().rename(columns={0:'number_publication'})
fund_author_full_names_list_count_2007['full_name'] = fund_author_full_names_list_count_2007['index'].str.lower()
fund_author_full_names_list_count_2007

,index,number_publication,full_name
0,"Bavari, Sina",46,"bavari, sina"
1,"Geisbert, Thomas W.",40,"geisbert, thomas w."
2,"Feldmann, Heinz",34,"feldmann, heinz"
3,"Basler, Christopher F.",31,"basler, christopher f."
4,"Becker, Stephan",26,"becker, stephan"
...,...,...,...
3139,"Kwesiga, Benon",1,"kwesiga, benon"
3140,"Kofman, Aaron",1,"kofman, aaron"
3141,"Masiira, Ben",1,"masiira, ben"
3142,"Brown, Shelley",1,"brown, shelley"


In [ ]:
fund_author_full_names_list_count_2007_concat = fund_author_full_names_list_count_2007.merge(all_author_full_names_list_and_gender, on = 'full_name', how='left')
fund_author_full_names_list_count_2007_concat

,index,number_publication,full_name,Unnamed: 0,0,firstname,ga_first_name,ga_gender,ga_accuracy,ga_samples,Unnamed: 8
0,"Bavari, Sina",46,"bavari, sina",0,54,Sina,Sina,male,63.0,5878.0,0
1,"Geisbert, Thomas W.",40,"geisbert, thomas w.",1,47,Thomas W.,Thomas W.,male,100.0,24.0,0
2,"Feldmann, Heinz",34,"feldmann, heinz",2,44,Heinz,Heinz,male,98.0,3977.0,0
3,"Basler, Christopher F.",31,"basler, christopher f.",6,31,Christopher F.,Christopher,male,99.0,55041.0,0
4,"Becker, Stephan",26,"becker, stephan",4,32,Stephan,Stephan,male,99.0,19941.0,0
...,...,...,...,...,...,...,...,...,...,...,...
3146,"Kwesiga, Benon",1,"kwesiga, benon",3794,1,Benon,Benon,male,99.0,91.0,0
3147,"Kofman, Aaron",1,"kofman, aaron",3795,1,Aaron,Aaron,male,99.0,36714.0,0
3148,"Masiira, Ben",1,"masiira, ben",3796,1,Ben,Ben,male,97.0,73526.0,0
3149,"Brown, Shelley",1,"brown, shelley",3797,1,Shelley,Shelley,female,98.0,4087.0,0


In [ ]:
fund_author_full_names_list_count_2007_concat['ga_gender'].value_counts()

male            1861
female          1158
unclassified     115
unknown           17
Name: ga_gender, dtype: int64

In [ ]:
fund_author_full_names_list_count_2007_concat['ga_gender'].value_counts(normalize=True)

male            0.590606
female          0.367502
unclassified    0.036496
unknown         0.005395
Name: ga_gender, dtype: float64

In [ ]:
# fund_data_author_lower_dummies = data_author_lower_dummies.reset_index().loc[data_author_lower_dummies.reset_index()['funding_yes'] == 1]
# fund_data_author_lower_dummies.shape

(577, 13)

In [ ]:
# fund_year_gender = fund_data_author_lower_dummies.reset_index()[['UT (Unique WOS ID)','Publication Year','female','male','unclassified','unknown','number_of_author']
#                                                      ].groupby(['Publication Year']).agg({'female': 'sum', 'male': 'sum',
#                                                                                           'unclassified': 'sum', 'unknown': 'sum', 'number_of_author': 'sum',
#                                                                                           'UT (Unique WOS ID)': 'count'})

# fund_year_gender['female_to_male_ratio'] = fund_year_gender['female']/fund_year_gender['male']
# fund_year_gender['publication_to_authors_number_ratio'] = fund_year_gender['number_of_author']/fund_year_gender['UT (Unique WOS ID)']

# for col in ['female','male','unclassified','unknown']:
#     fund_year_gender['%_'+col] = fund_year_gender[col]/fund_year_gender['number_of_author']*100

# # #save data
# fund_year_gender.to_csv(f'../TABLES/year_gender_fund_count.csv')
# fund_year_gender

In [ ]:
# fund_year_gender = fund_data_author_lower_dummies[['UT (Unique WOS ID)','Publication Year','female','male','unclassified','unknown','number_of_author']].groupby(['Publication Year']).sum()#.drop('funding_yes',axis=1)
# fund_year_gender['female_to_male_ratio'] = fund_year_gender['female']/fund_year_gender['male']

# for col in ['female','male','unclassified','unknown']:
#     fund_year_gender['%_'+col] = fund_year_gender[col]/fund_year_gender['number_of_author']*100

# # #save data
# fund_year_gender.to_csv(f'../DATA_PREPROCESSING/year_gender_fund_count.csv')
# fund_year_gender

## non funding

In [ ]:
from functools import reduce

#Import data
data = pd.read_excel(f'D:\MARBURG VIRUS DISEASES/data/WOSCC/woscc_bon_23_02_verif_on_wos.xls')[['UT (Unique WOS ID)',
                                                                                                'Publication Year',
                                                                                                'Funding Orgs',
                                                                                                'Author Full Names']];
# limit data to after 2007
data_2007 = data.reset_index()[data.reset_index()['Publication Year'] > 2006]
data_2007.head()

# create a columns of funding yes or no
data_2007['funding_yes'] = data_2007['Funding Orgs'].apply(lambda x: 1 if not pd.isnull(x) else 0)#.to_frame().drop('Funding Orgs', axis=1)

data_2007 = data_2007.reset_index().loc[data_2007.reset_index()['funding_yes'] == 0]

# set accession number as index
data_2007 = data_2007.set_index(['UT (Unique WOS ID)','Publication Year','Funding Orgs','funding_yes'])

#
df_expand_2007 = data_2007['Author Full Names'].str.split('; ', expand=True)

# extract all the cells data
author_full_names_list_2007 = df_expand_2007.stack().explode().droplevel(0).to_frame().reset_index()#.drop('index', axis=1)

###
no_fund_author_full_names_list_count_2007 =author_full_names_list_2007[0].value_counts().to_frame()

## extract firstname
# author_full_names_list_count_2007 = author_full_names_list_count_2007.reset_index()
# author_full_names_list_count_2007['firstname'] = author_full_names_list_count_2007['index'].str.split(', ', expand=True)[[1]]

 #save data
no_fund_author_full_names_list_count_2007.to_csv(f'../DATA_PREPROCESSING/author_full_names_list_count_and_firstname_2007_no_fund.csv')

#print
no_fund_author_full_names_list_count_2007

,0
"Feldmann, Heinz",8
"Bausch, Daniel G.",6
"Nichol, Stuart T.",6
"Becker, Stephan",6
"Geisbert, Thomas W.",6
...,...
"Pyankov, Oleg V.",1
"Camara, Jacob",1
"Sylla, Bakary",1
"Agafonov, Alexander P.",1


In [ ]:
no_fund_author_full_names_list_count_2007 = no_fund_author_full_names_list_count_2007.reset_index().rename(columns={0:'number_publication'})
no_fund_author_full_names_list_count_2007['full_name'] = no_fund_author_full_names_list_count_2007['index'].str.lower()
no_fund_author_full_names_list_count_2007

,index,number_publication,full_name
0,"Feldmann, Heinz",8,"feldmann, heinz"
1,"Bausch, Daniel G.",6,"bausch, daniel g."
2,"Nichol, Stuart T.",6,"nichol, stuart t."
3,"Becker, Stephan",6,"becker, stephan"
4,"Geisbert, Thomas W.",6,"geisbert, thomas w."
...,...,...,...
623,"Pyankov, Oleg V.",1,"pyankov, oleg v."
624,"Camara, Jacob",1,"camara, jacob"
625,"Sylla, Bakary",1,"sylla, bakary"
626,"Agafonov, Alexander P.",1,"agafonov, alexander p."


In [ ]:
no_fund_author_full_names_list_count_2007_concat = no_fund_author_full_names_list_count_2007.merge(all_author_full_names_list_and_gender, on = 'full_name', how='left')
no_fund_author_full_names_list_count_2007_concat

,index,number_publication,full_name,Unnamed: 0,0,firstname,ga_first_name,ga_gender,ga_accuracy,ga_samples,Unnamed: 8
0,"Feldmann, Heinz",8,"feldmann, heinz",2,44,Heinz,Heinz,male,98.0,3977.0,0
1,"Bausch, Daniel G.",6,"bausch, daniel g.",80,8,Daniel G.,Daniel G.,male,100.0,24.0,0
2,"Nichol, Stuart T.",6,"nichol, stuart t.",3,33,Stuart T.,Stuart,male,99.0,12253.0,0
3,"Becker, Stephan",6,"becker, stephan",4,32,Stephan,Stephan,male,99.0,19941.0,0
4,"Geisbert, Thomas W.",6,"geisbert, thomas w.",1,47,Thomas W.,Thomas W.,male,100.0,24.0,0
...,...,...,...,...,...,...,...,...,...,...,...
624,"Pyankov, Oleg V.",1,"pyankov, oleg v.",4033,1,Oleg V.,Oleg V.,male,100.0,6.0,0
625,"Camara, Jacob",1,"camara, jacob",903,2,Jacob,Jacob,male,99.0,25605.0,0
626,"Sylla, Bakary",1,"sylla, bakary",4045,1,Bakary,Bakary,male,99.0,1775.0,0
627,"Agafonov, Alexander P.",1,"agafonov, alexander p.",4047,1,Alexander P.,Alexander P.,male,100.0,4.0,0


In [ ]:
no_fund_author_full_names_list_count_2007_concat['ga_gender'].value_counts()

male            393
female          204
unclassified     31
unknown           1
Name: ga_gender, dtype: int64

In [ ]:
no_fund_author_full_names_list_count_2007_concat['ga_gender'].value_counts(normalize=True)

male            0.624801
female          0.324324
unclassified    0.049285
unknown         0.001590
Name: ga_gender, dtype: float64

In [ ]:
# no_fund_data_author_lower_dummies = data_author_lower_dummies.reset_index().loc[data_author_lower_dummies.reset_index()['funding_yes'] == 0]
# no_fund_data_author_lower_dummies.shape

In [ ]:
# no_fund_year_gender = no_fund_data_author_lower_dummies.reset_index()[['UT (Unique WOS ID)','Publication Year','female','male','unclassified','unknown','number_of_author']
#                                                      ].groupby(['Publication Year']).agg({'female': 'sum', 'male': 'sum',
#                                                                                           'unclassified': 'sum', 'unknown': 'sum', 'number_of_author': 'sum',
#                                                                                           'UT (Unique WOS ID)': 'count'})

# no_fund_year_gender['female_to_male_ratio'] = no_fund_year_gender['female']/no_fund_year_gender['male']
# no_fund_year_gender['publication_to_authors_number_ratio'] = no_fund_year_gender['number_of_author']/no_fund_year_gender['UT (Unique WOS ID)']

# for col in ['female','male','unclassified','unknown']:
#     no_fund_year_gender['%_'+col] = no_fund_year_gender[col]/no_fund_year_gender['number_of_author']*100

# # #save data
# no_fund_year_gender.to_csv(f'../TABLES/year_gender_no_fund_count.csv')
# no_fund_year_gender

In [ ]:
# no_fund_year_gender = no_fund_data_author_lower_dummies[['UT (Unique WOS ID)','Publication Year','female','male','unclassified','unknown','number_of_author']].groupby(['Publication Year']).sum()#.drop('funding_yes',axis=1)
# no_fund_year_gender['female_to_male_ratio'] = no_fund_year_gender['female']/no_fund_year_gender['male']

# for col in ['female','male','unclassified','unknown']:
#     no_fund_year_gender['%_'+col] = no_fund_year_gender[col]/no_fund_year_gender['number_of_author']*100

# # #save data
# no_fund_year_gender.to_csv(f'../DATA_PREPROCESSING/year_gender_no_fund_count.csv')
# no_fund_year_gender

## significance

In [ ]:
### compraring the female proportion

In [ ]:
# limit data to after 2007
all_author_lower_dummies_2007 = data_author_lower_dummies.reset_index()[data_author_lower_dummies.reset_index()['Publication Year'] > 2006]
all_author_lower_dummies_2007.head()

,UT (Unique WOS ID),Publication Year,Funding Orgs,funding_yes,female,male,unclassified,unknown,number_of_author,%_female,%_male,%_unclass_name,female_to_male_ratio
0,WOS:000904661800001,2023.0,NaN,0,2.0,7.0,3.0,1.0,13.0,15.384615,53.846154,23.076923,0.285714
1,WOS:000921279800001,2023.0,National Institutes of Health (NIH) [U19AI1427...,1,2.0,9.0,0.0,0.0,11.0,18.181818,81.818182,0.000000,0.222222
2,WOS:000909001200001,2023.0,Russian Science Foundation; [22-13-00036],1,0.0,1.0,0.0,0.0,1.0,0.000000,100.000000,0.000000,0.000000
3,WOS:000777045900001,2022.0,National Research Foundation of Korea (NRF) - ...,1,1.0,12.0,1.0,0.0,14.0,7.142857,85.714286,7.142857,0.083333
4,WOS:000849602200001,2022.0,NaN,0,0.0,6.0,0.0,0.0,6.0,0.000000,100.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
714,WOS:000244948100002,2007.0,NaN,0,0.0,2.0,0.0,0.0,2.0,0.000000,100.000000,0.000000,0.000000
715,WOS:000243871100009,2007.0,NCI NIH HHS [CA 092459] Funding Source: Medlin...,1,4.0,2.0,0.0,1.0,7.0,57.142857,28.571429,0.000000,2.000000
716,WOS:000244234800016,2007.0,NaN,0,0.0,2.0,0.0,0.0,2.0,0.000000,100.000000,0.000000,0.000000
717,WOS:000243692500011,2007.0,"NIAID NIH HHS [R01 AI051292, R21 AI056118, AI0...",1,3.0,21.0,0.0,0.0,24.0,12.500000,87.500000,0.000000,0.142857


In [ ]:
fund_all_author_gender_2007 = all_author_lower_dummies_2007.loc[all_author_lower_dummies_2007['funding_yes'] == 1]
no_fund_all_author_gender_2007 = all_author_lower_dummies_2007.loc[all_author_lower_dummies_2007['funding_yes'] == 0]

In [ ]:
all_author_lower_dummies_2007['%_female'].describe()

count    714.000000
mean      33.503799
std       24.045653
min        0.000000
25%       16.666667
50%       33.333333
75%       50.000000
max      100.000000
Name: %_female, dtype: float64

In [ ]:
fund_all_author_gender_2007['%_female'].describe()

count    563.000000
mean      34.419170
std       22.323173
min        0.000000
25%       20.000000
50%       33.333333
75%       50.000000
max      100.000000
Name: %_female, dtype: float64

In [ ]:
no_fund_all_author_gender_2007['%_female'].describe()

count    151.000000
mean      30.090860
std       29.435033
min        0.000000
25%        0.000000
50%       27.777778
75%       50.000000
max      100.000000
Name: %_female, dtype: float64

In [ ]:
test_sig(fund_all_author_gender_2007['%_female'], no_fund_all_author_gender_2007['%_female'])

mann-whitney


MannwhitneyuResult(statistic=47969.0, pvalue=0.014832120105879979)

# first authors gender

In [ ]:
first_author_gender = data_author_lower[0].to_frame().rename(columns={0:'ga_gender'})
#save data
first_author_gender.to_csv(f'../DATA_PREPROCESSING/first_author_gender.csv')
first_author_gender

,,,,ga_gender
UT (Unique WOS ID),Publication Year,Funding Orgs,funding_yes,
WOS:000904661800001,2023.0,NaN,0,male
WOS:000921279800001,2023.0,National Institutes of Health (NIH) [U19AI142785]; NIH [UC7AI094660],1,male
WOS:000909001200001,2023.0,Russian Science Foundation; [22-13-00036],1,male
WOS:000777045900001,2022.0,National Research Foundation of Korea (NRF) - Ministry of Education [NRF-2020R1I1A2066868]; National Research Foundation of Korea (NRF) - Korean government (MSIT) [2020R1A5A2019413],1,male
WOS:000849602200001,2022.0,NaN,0,male
...,...,...,...,...
WOS:A1968B358900049,1968.0,NaN,0,unclassified
WOS:A1968A855300001,1968.0,NaN,0,unclassified
WOS:A1968A855300006,1968.0,NaN,0,unclassified


In [ ]:
# limit data to after 2007
first_author_gender_2007 = first_author_gender.reset_index()[first_author_gender.reset_index()['Publication Year'] > 2006]
first_author_gender_2007#.head()

,UT (Unique WOS ID),Publication Year,Funding Orgs,funding_yes,ga_gender
0,WOS:000904661800001,2023.0,NaN,0,male
1,WOS:000921279800001,2023.0,National Institutes of Health (NIH) [U19AI1427...,1,male
2,WOS:000909001200001,2023.0,Russian Science Foundation; [22-13-00036],1,male
3,WOS:000777045900001,2022.0,National Research Foundation of Korea (NRF) - ...,1,male
4,WOS:000849602200001,2022.0,NaN,0,male
...,...,...,...,...,...
714,WOS:000244948100002,2007.0,NaN,0,male
715,WOS:000243871100009,2007.0,NCI NIH HHS [CA 092459] Funding Source: Medlin...,1,male
716,WOS:000244234800016,2007.0,NaN,0,male
717,WOS:000243692500011,2007.0,"NIAID NIH HHS [R01 AI051292, R21 AI056118, AI0...",1,male


In [ ]:
first_author_gender_2007['ga_gender'].value_counts()

male            406
female          287
unclassified     19
unknown           2
Name: ga_gender, dtype: int64

In [ ]:
first_author_gender_2007['ga_gender'].value_counts(normalize=True)

male            0.568627
female          0.401961
unclassified    0.026611
unknown         0.002801
Name: ga_gender, dtype: float64

## funding

In [ ]:
fund_first_author_gender_2007 = first_author_gender_2007.loc[first_author_gender_2007['funding_yes'] == 1]

In [ ]:
fund_first_author_gender_2007['ga_gender'].value_counts()

male            305
female          243
unclassified     13
unknown           2
Name: ga_gender, dtype: int64

In [ ]:
fund_first_author_gender_2007['ga_gender'].value_counts(normalize=True)

male            0.541741
female          0.431616
unclassified    0.023091
unknown         0.003552
Name: ga_gender, dtype: float64

In [ ]:
fund_first_author_gender_2007.shape

(563, 5)

## no funding

In [ ]:
no_fund_first_author_gender_2007 = first_author_gender_2007.loc[first_author_gender_2007['funding_yes'] == 0]

In [ ]:
no_fund_first_author_gender_2007['ga_gender'].value_counts()

male            101
female           44
unclassified      6
Name: ga_gender, dtype: int64

In [ ]:
no_fund_first_author_gender_2007['ga_gender'].value_counts(normalize=True)

male            0.668874
female          0.291391
unclassified    0.039735
Name: ga_gender, dtype: float64

In [ ]:
no_fund_first_author_gender_2007.shape

(151, 5)

## significance

In [ ]:
fund_first_author_gender_2007[fund_first_author_gender_2007['ga_gender'].isin(['male', 'female'])].replace({'male': 1, 'female': 0})

,UT (Unique WOS ID),Publication Year,Funding Orgs,funding_yes,ga_gender
1,WOS:000921279800001,2023.0,National Institutes of Health (NIH) [U19AI1427...,1,1
2,WOS:000909001200001,2023.0,Russian Science Foundation; [22-13-00036],1,1
3,WOS:000777045900001,2022.0,National Research Foundation of Korea (NRF) - ...,1,1
6,WOS:000905136600002,2022.0,Intramural Research Program of the Vaccine Res...,1,0
10,WOS:000896010800003,2022.0,"National Institutes of Health [R33AI100190, AI...",1,0
...,...,...,...,...,...
708,WOS:000244881900014,2007.0,NATIONAL INSTITUTE OF ALLERGY AND INFECTIOUS D...,1,0
709,WOS:000244881900012,2007.0,NIDCD NIH HHS [R01 DC003299] Funding Source: M...,1,0
712,WOS:000249426800015,2007.0,"NIAID NIH HHS [R21 AI063436, R21 AI063436-01A1...",1,0
715,WOS:000243871100009,2007.0,NCI NIH HHS [CA 092459] Funding Source: Medlin...,1,1


In [ ]:
fund_first_author_gender_2007_test = fund_first_author_gender_2007[fund_first_author_gender_2007['ga_gender'].isin(['male', 'female'])].replace({'male': 1, 'female': 0})['ga_gender']
no_fund_first_author_gender_2007_test = no_fund_first_author_gender_2007[no_fund_first_author_gender_2007['ga_gender'].isin(['male', 'female'])].replace({'male': 1, 'female': 0})['ga_gender']

In [ ]:
test_sig(fund_first_author_gender_2007_test, no_fund_first_author_gender_2007_test)

mann-whitney


MannwhitneyuResult(statistic=34168.5, pvalue=0.002361160480953111)

In [ ]:
fund_first_author_gender_2007_test.shape

(548, 5)

In [ ]:
fund_first_author_gender_2007_test['ga_gender'].value_counts()

1    305
0    243
Name: ga_gender, dtype: int64

In [ ]:
no_fund_first_author_gender_2007_test.shape

(145, 5)

In [ ]:
no_fund_first_author_gender_2007_test['ga_gender'].value_counts()

1    101
0     44
Name: ga_gender, dtype: int64

# last authors gender

## all

In [ ]:
# create a columns of funding yes or no
data['funding_yes'] = data['Funding Orgs'].apply(lambda x: 1 if not pd.isnull(x) else 0)
# data = data.loc[data.reset_index()['funding_yes'] == 0]

data = data.set_index(['UT (Unique WOS ID)','Publication Year','Funding Orgs','funding_yes'])
last_author_gender = data['Author Full Names'].map(lambda x: x.split('; ')[-1]).str.lower().to_frame()
last_author_gender['gender'] = last_author_gender['Author Full Names'].map(name_gender_keys_values)

#save data
last_author_gender.to_csv(f'../DATA_PREPROCESSING/last_author_gender.csv')
last_author_gender

,,,,Author Full Names,gender
UT (Unique WOS ID),Publication Year,Funding Orgs,funding_yes,,
WOS:000904661800001,2023.0,NaN,0,"xia, xianzhu",female
WOS:000921279800001,2023.0,National Institutes of Health (NIH) [U19AI142785]; NIH [UC7AI094660],1,"heyes, james",male
WOS:000909001200001,2023.0,Russian Science Foundation; [22-13-00036],1,"moskalik, mikhail yu.",male
WOS:000777045900001,2022.0,National Research Foundation of Korea (NRF) - Ministry of Education [NRF-2020R1I1A2066868]; National Research Foundation of Korea (NRF) - Korean government (MSIT) [2020R1A5A2019413],1,"hassan, mohammad mahmudul",male
WOS:000849602200001,2022.0,NaN,0,"dhama, kuldeep",male
...,...,...,...,...,...
WOS:A1968B358900049,1968.0,NaN,0,strickla.m,unclassified
WOS:A1968A855300001,1968.0,NaN,0,"baltzer, g",unclassified
WOS:A1968A855300006,1968.0,NaN,0,"slenczka, w",unclassified


In [ ]:
# limit data to after 2007
last_author_gender_2007 = last_author_gender.reset_index()
last_author_gender_2007 = last_author_gender_2007[last_author_gender_2007['Publication Year'] > 2006]
last_author_gender_2007#.head()

,UT (Unique WOS ID),Publication Year,Funding Orgs,funding_yes,Author Full Names,gender
0,WOS:000904661800001,2023.0,NaN,0,"xia, xianzhu",female
1,WOS:000921279800001,2023.0,National Institutes of Health (NIH) [U19AI1427...,1,"heyes, james",male
2,WOS:000909001200001,2023.0,Russian Science Foundation; [22-13-00036],1,"moskalik, mikhail yu.",male
3,WOS:000777045900001,2022.0,National Research Foundation of Korea (NRF) - ...,1,"hassan, mohammad mahmudul",male
4,WOS:000849602200001,2022.0,NaN,0,"dhama, kuldeep",male
...,...,...,...,...,...,...
714,WOS:000244948100002,2007.0,NaN,0,"mohamadzadeh, mansour",male
715,WOS:000243871100009,2007.0,NCI NIH HHS [CA 092459] Funding Source: Medlin...,1,"rong, lijun",female
716,WOS:000244234800016,2007.0,NaN,0,"geisbert, thomas w.",male
717,WOS:000243692500011,2007.0,"NIAID NIH HHS [R01 AI051292, R21 AI056118, AI0...",1,"lipkin, w. ian",male


In [ ]:
last_author_gender_2007['gender'].value_counts()

male            562
female          138
unclassified     12
unknown           2
Name: gender, dtype: int64

In [ ]:
last_author_gender_2007['gender'].value_counts(normalize=True)

male            0.787115
female          0.193277
unclassified    0.016807
unknown         0.002801
Name: gender, dtype: float64

## funding

In [ ]:
fund_last_author_gender_2007 = last_author_gender_2007.loc[last_author_gender_2007['funding_yes'] == 1]

In [ ]:
fund_last_author_gender_2007['gender'].value_counts()

male            446
female          109
unclassified      6
unknown           2
Name: gender, dtype: int64

In [ ]:
fund_last_author_gender_2007['gender'].value_counts(normalize=True)

male            0.792185
female          0.193606
unclassified    0.010657
unknown         0.003552
Name: gender, dtype: float64

In [ ]:
fund_last_author_gender_2007.shape

(563, 6)

## no funding

In [ ]:
no_fund_last_author_gender_2007 = last_author_gender_2007.loc[last_author_gender_2007['funding_yes'] == 0]

In [ ]:
no_fund_last_author_gender_2007['gender'].value_counts()

male            116
female           29
unclassified      6
Name: gender, dtype: int64

In [ ]:
no_fund_last_author_gender_2007['gender'].value_counts(normalize=True)

male            0.768212
female          0.192053
unclassified    0.039735
Name: gender, dtype: float64

In [ ]:
no_fund_last_author_gender_2007.shape

(151, 6)

In [ ]:
last_author_gender_2007

,UT (Unique WOS ID),Publication Year,Funding Orgs,funding_yes,Author Full Names,gender
0,WOS:000904661800001,2023.0,NaN,0,"xia, xianzhu",female
1,WOS:000921279800001,2023.0,National Institutes of Health (NIH) [U19AI1427...,1,"heyes, james",male
2,WOS:000909001200001,2023.0,Russian Science Foundation; [22-13-00036],1,"moskalik, mikhail yu.",male
3,WOS:000777045900001,2022.0,National Research Foundation of Korea (NRF) - ...,1,"hassan, mohammad mahmudul",male
4,WOS:000849602200001,2022.0,NaN,0,"dhama, kuldeep",male
...,...,...,...,...,...,...
714,WOS:000244948100002,2007.0,NaN,0,"mohamadzadeh, mansour",male
715,WOS:000243871100009,2007.0,NCI NIH HHS [CA 092459] Funding Source: Medlin...,1,"rong, lijun",female
716,WOS:000244234800016,2007.0,NaN,0,"geisbert, thomas w.",male
717,WOS:000243692500011,2007.0,"NIAID NIH HHS [R01 AI051292, R21 AI056118, AI0...",1,"lipkin, w. ian",male


## significance

In [ ]:
fund_last_author_gender_2007[fund_last_author_gender_2007['gender'].isin(['male', 'female'])].replace({'male': 1, 'female': 0})

,UT (Unique WOS ID),Publication Year,Funding Orgs,funding_yes,Author Full Names,gender
1,WOS:000921279800001,2023.0,National Institutes of Health (NIH) [U19AI1427...,1,"heyes, james",1
2,WOS:000909001200001,2023.0,Russian Science Foundation; [22-13-00036],1,"moskalik, mikhail yu.",1
3,WOS:000777045900001,2022.0,National Research Foundation of Korea (NRF) - ...,1,"hassan, mohammad mahmudul",1
6,WOS:000905136600002,2022.0,Intramural Research Program of the Vaccine Res...,1,"sullivan, nancy j.",0
10,WOS:000896010800003,2022.0,"National Institutes of Health [R33AI100190, AI...",1,"gehrke, lee",1
...,...,...,...,...,...,...
708,WOS:000244881900014,2007.0,NATIONAL INSTITUTE OF ALLERGY AND INFECTIOUS D...,1,"bavari, sina",1
709,WOS:000244881900012,2007.0,NIDCD NIH HHS [R01 DC003299] Funding Source: M...,1,"becker, stephan",1
712,WOS:000249426800015,2007.0,"NIAID NIH HHS [R21 AI063436, R21 AI063436-01A1...",1,"muehlberger, elke",0
715,WOS:000243871100009,2007.0,NCI NIH HHS [CA 092459] Funding Source: Medlin...,1,"rong, lijun",0


In [ ]:
fund_last_author_gender_2007_test = fund_last_author_gender_2007[fund_last_author_gender_2007['gender'].isin(['male', 'female'])].replace({'male': 1, 'female': 0})['gender']
no_fund_last_author_gender_2007_test = no_fund_last_author_gender_2007[no_fund_last_author_gender_2007['gender'].isin(['male', 'female'])].replace({'male': 1, 'female': 0})['gender']

In [ ]:
test_sig(fund_last_author_gender_2007_test, no_fund_last_author_gender_2007_test)

mann-whitney


MannwhitneyuResult(statistic=40382.5, pvalue=0.9229519490866986)

# significance

In [ ]:
# limit data to after 2007
data_author_lower_dummies_2007 = data_author_lower_dummies.reset_index()[data_author_lower_dummies.reset_index()['Publication Year'] > 2006]
data_author_lower_dummies_2007.head()

,UT (Unique WOS ID),Publication Year,Funding Orgs,funding_yes,female,male,unclassified,unknown,number_of_author,%_female,%_male,%_unclass_name,female_to_male_ratio
0,WOS:000904661800001,2023.0,NaN,0,2.0,7.0,3.0,1.0,13.0,15.384615,53.846154,23.076923,0.285714
1,WOS:000921279800001,2023.0,National Institutes of Health (NIH) [U19AI1427...,1,2.0,9.0,0.0,0.0,11.0,18.181818,81.818182,0.000000,0.222222
2,WOS:000909001200001,2023.0,Russian Science Foundation; [22-13-00036],1,0.0,1.0,0.0,0.0,1.0,0.000000,100.000000,0.000000,0.000000
3,WOS:000777045900001,2022.0,National Research Foundation of Korea (NRF) - ...,1,1.0,12.0,1.0,0.0,14.0,7.142857,85.714286,7.142857,0.083333
4,WOS:000849602200001,2022.0,NaN,0,0.0,6.0,0.0,0.0,6.0,0.000000,100.000000,0.000000,0.000000


In [ ]:
data_author_lower_dummies_2007.describe()

,Publication Year,funding_yes,female,male,unclassified,unknown,number_of_author,%_female,%_male,%_unclass_name,female_to_male_ratio
count,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,706.000000
mean,2015.537815,0.788515,2.810924,5.207283,0.218487,0.030812,8.267507,33.503799,63.371826,2.791472,inf
std,4.403473,0.408647,2.590162,4.698381,1.025561,0.188454,6.549617,24.045653,24.798693,12.591060,NaN
min,2007.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,2012.000000,1.000000,1.000000,2.000000,0.000000,0.000000,4.000000,16.666667,50.000000,0.000000,0.205556
50%,2016.000000,1.000000,2.000000,4.000000,0.000000,0.000000,7.000000,33.333333,63.636364,0.000000,0.500000
75%,2019.000000,1.000000,4.000000,7.000000,0.000000,0.000000,11.000000,50.000000,80.000000,0.000000,1.000000
max,2023.000000,1.000000,18.000000,58.000000,17.000000,2.000000,76.000000,100.000000,100.000000,100.000000,inf


In [ ]:
data_author_lower_dummies_2007['female_to_male_ratio'].value_counts().to_frame()

,female_to_male_ratio
0.000000,119
1.000000,83
0.500000,47
0.333333,35
0.666667,28
...,...
0.307692,1
1.600000,1
0.230769,1
0.277778,1


In [ ]:
data_author_lower_dummies['female_to_male_ratio'].count()

853

In [ ]:
data_author_lower_dummies_2007['female_to_male_ratio'].value_counts().to_frame()['female_to_male_ratio'].sum()

706

In [ ]:
data_author_lower_dummies_2007.loc[data_author_lower_dummies_2007['funding_yes'] == 0].describe()

,Publication Year,funding_yes,female,male,unclassified,unknown,number_of_author,%_female,%_male,%_unclass_name,female_to_male_ratio
count,151.000000,151.0,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,148.000000
mean,2013.556291,0.0,1.516556,3.344371,0.211921,0.006623,5.079470,30.090860,65.573369,4.284828,inf
std,5.142808,0.0,1.656721,2.754503,0.744842,0.081379,3.539347,29.435033,30.446068,16.543160,NaN
min,2007.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,2009.000000,0.0,0.000000,1.000000,0.000000,0.000000,2.000000,0.000000,50.000000,0.000000,0.000000
50%,2013.000000,0.0,1.000000,2.000000,0.000000,0.000000,4.000000,27.777778,66.666667,0.000000,0.464286
75%,2018.000000,0.0,2.000000,5.000000,0.000000,0.000000,7.000000,50.000000,100.000000,0.000000,1.000000
max,2023.000000,0.0,9.000000,14.000000,6.000000,1.000000,19.000000,100.000000,100.000000,100.000000,inf


In [ ]:
data_author_lower_dummies_2007.loc[data_author_lower_dummies_2007['funding_yes'] == 1].describe()

,Publication Year,funding_yes,female,male,unclassified,unknown,number_of_author,%_female,%_male,%_unclass_name,female_to_male_ratio
count,563.000000,563.0,563.000000,563.000000,563.000000,563.000000,563.000000,563.000000,563.000000,563.000000,558.00
mean,2016.069272,1.0,3.158082,5.706927,0.220249,0.037300,9.122558,34.419170,62.781359,2.390945,inf
std,4.025342,0.0,2.684783,4.979789,1.089165,0.207582,6.900480,22.323173,23.046629,11.283851,NaN
min,2007.000000,1.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.00
25%,2013.000000,1.0,1.000000,3.000000,0.000000,0.000000,5.000000,20.000000,50.000000,0.000000,0.25
50%,2017.000000,1.0,3.000000,5.000000,0.000000,0.000000,8.000000,33.333333,63.636364,0.000000,0.50
75%,2019.000000,1.0,5.000000,7.000000,0.000000,0.000000,12.000000,50.000000,77.777778,0.000000,1.00
max,2023.000000,1.0,18.000000,58.000000,17.000000,2.000000,76.000000,100.000000,100.000000,100.000000,inf


In [ ]:
# data_author_lower_dummies_2007

In [ ]:
# data_author_lower['number_of_author'] = data_author_lower.count(axis='columns')
# data_author_lower['number_of_male'] = (data_author_lower == 'male').sum(axis=1)
# data_author_lower['number_of_female'] = (data_author_lower == 'female').sum(axis=1)
# data_author_lower['number_of_unknown_or_unclassified'] = ((data_author_lower == '/') | (data_author_lower == 'unknown')).sum(axis=1)
# data_author_lower['proportion_unclass_name'] = data_author_lower['number_of_unknown_or_unclassified']/data_author_lower['number_of_author']*100
# data_author_lower['female_to_male_ratio'] = data_author_lower['number_of_female']/data_author_lower['number_of_male']
# data_author_lower['first_author_gender'] = data_author_lower[0]
# # data_author_lower['first_author_gender'] =

# #save data
# data_author_lower.to_csv(f'../DATA_PREPROCESSING/author_gender_list_count.csv')
# data_author_lower

In [ ]:
# data_author_lower['female_to_male_ratio'].count()